In [27]:
import hoomd
import flowermd
import numpy as np
import matplotlib as plt
import mbuild as mb
import warnings
warnings.filterwarnings('ignore')
from flowermd.base import Molecule
from flowermd.library import FF_from_file

from flowermd.base import Lattice
from cmeutils.visualize import FresnelGSD
from flowermd.base import Simulation

In [28]:
def espaloma_mol(file_path):
    mol = mb.load(file_path)
    for p in mol.particles():
        p.name = f"_{p.name}"
    return mol

In [29]:
system_file = '../cy5.mol2'
ff_filepath =  "../cy5.xml"

In [30]:
esp_mol = espaloma_mol(system_file)
cy5 = Molecule(num_mols = 2, compound = esp_mol)

In [31]:
cy5.molecules[0].visualize()

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [32]:
cy5_ff = FF_from_file(forcefield_files=ff_filepath)
system=Lattice(molecules=cy5, x=1,y=1, n=1)

In [33]:
system.box

Box: Lx=2.149367, Ly=1.980101, Lz=1.946039, xy=0.000000, xz=0.000000, yz=0.000000, 

In [34]:
system.apply_forcefield(
    r_cut=2.5, 
    force_field=cy5_ff, 
    auto_scale=True, 
    scale_charges=True,
    remove_hydrogens=True, #check this
    remove_charges=False) #check this 

In [35]:
snapshot= system.hoomd_snapshot
snapshot.configuration.box *= 3
snapshot.configuration.box

array([18.5255822 , 17.06666374, 16.77308038,  0.        ,  0.        ,
        0.        ])

In [36]:
hoomd_forces = system.hoomd_forcefield
hoomd_forces

In [37]:
working_forces = []
for f in hoomd_forces:
    if not isinstance(f, hoomd.md.special_pair.LJ):
        working_forces.append(f)
working_forces

In [38]:
sim=Simulation(initial_state=snapshot, forcefield=working_forces, reference_values=system.reference_values, gsd_write_freq=100,log_write_freq=100)

Initializing simulation state from a gsd.hoomd.Frame.


In [39]:
lj_force = hoomd_forces[4]

dict(lj_force.params)
#here we see the parameters of the force field t0 and k

{}

In [40]:
sim.box_lengths_reduced

array([18.52558136, 17.06666374, 16.77308083])

In [41]:
system.system.visualize()

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [ ]:
sim.run_NVT(n_steps=2000000, kT=1.5, tau_kt=0.01)

*Warning*: charge.pppm: RMS error of 0.210864 is probably too high! 0.210864 0.21085


Step 100 of 2000000; TPS: 243.71; ETA: 2.0 hours, 17.0 minutes
Step 200 of 2000000; TPS: 453.26; ETA: 1.0 hours, 14.0 minutes
Step 300 of 2000000; TPS: 634.75; ETA: 52.5 minutes
Step 400 of 2000000; TPS: 789.55; ETA: 42.2 minutes
Step 500 of 2000000; TPS: 907.46; ETA: 36.7 minutes
Step 600 of 2000000; TPS: 1026.94; ETA: 32.4 minutes
Step 700 of 2000000; TPS: 1142.52; ETA: 29.2 minutes
Step 800 of 2000000; TPS: 1255.79; ETA: 26.5 minutes
Step 900 of 2000000; TPS: 1360.9; ETA: 24.5 minutes
Step 1000 of 2000000; TPS: 1458.19; ETA: 22.8 minutes
Step 1100 of 2000000; TPS: 1549.62; ETA: 21.5 minutes
Step 1200 of 2000000; TPS: 1637.49; ETA: 20.3 minutes
Step 1300 of 2000000; TPS: 1710.8; ETA: 19.5 minutes
Step 1400 of 2000000; TPS: 1789.72; ETA: 18.6 minutes
Step 1500 of 2000000; TPS: 1865.44; ETA: 17.9 minutes
Step 1600 of 2000000; TPS: 1936.85; ETA: 17.2 minutes
Step 1700 of 2000000; TPS: 2005.14; ETA: 16.6 minutes
Step 1800 of 2000000; TPS: 2069.53; ETA: 16.1 minutes
Step 1900 of 2000000; 

In [ ]:
for writer in sim.operations.writers:
    if isinstance(writer, hoomd.write.GSD):
        writer.flush()

In [ ]:
sim_visualizer = FresnelGSD(gsd_file="trajectory.gsd", frame=0, view_axis=(1, 1, 1))
sim_visualizer.view()

In [ ]:
sim_visualizer.frame=-1
sim_visualizer.view()